In [1]:
import json
import os
os.chdir("..")

In [8]:
# Inputs

path = os.path.join(os.getcwd(), "utils", "799988a6cbeb", "Input.registry.file")
with open(path, "r") as file:
    input_config_file = json.loads(file.read())

input_config_mqtt = {}
model_name = "CarParkModel"

defined_external_names = ['SoC_Value']
defined_non_prediction_names =  ['P_PV']
defined_prediction_names =  []
external_names =  []

prediction_names = []
non_prediction_names = []
external_names = []
generic_names = []

def add_name_to_list(key):
    if key in defined_prediction_names:
        prediction_names.append(key)
    elif key in defined_non_prediction_names:
        non_prediction_names.append(key)
    elif key in defined_external_names:
        external_names.append(key)
    else:
        generic_names.append(key)

model_variables = {'Feasible_ESS_Decisions': {'type': 'Set', 'indexing': 'None'}, 'Feasible_VAC_Decisions': {'type': 'Set', 'indexing': 'None'}, 'Value_Index': {'type': 'Set', 'indexing': 'None'}, 'Value': {'type': 'Param', 'indexing': 'index'}, 'P_PV_Forecast': {'type': 'Param', 'indexing': 'None'}, 'Initial_ESS_SoC': {'type': 'Param', 'indexing': 'index'}, 'Initial_VAC_SoC': {'type': 'Param', 'indexing': 'index'}, 'Number_of_Parked_Cars': {'type': 'Param', 'indexing': 'None'}, 'Unit_Consumption_Assumption': {'type': 'Param', 'indexing': 'None'}, 'Unit_Drop_Penalty': {'type': 'Param', 'indexing': 'None'}, 'ESS_Capacity': {'type': 'Param', 'indexing': 'None'}, 'VAC_Capacity': {'type': 'Param', 'indexing': 'None'}, 'Behavior_Model_Index': {'type': 'Set', 'indexing': 'None'}, 'Behavior_Model': {'type': 'Param', 'indexing': 'None'}, 'dT': {'type': 'Param', 'indexing': 'None'}, 'Decision': {'type': 'Var', 'indexing': 'index'}, 'P_ESS': {'type': 'Var', 'indexing': 'None'}, 'P_VAC': {'type': 'Var', 'indexing': 'None'}, 'P_PV': {'type': 'Var', 'indexing': 'index'}, 'P_GRID': {'type': 'Var', 'indexing': 'None'}}

class Constants:

    ESS = "ESS"
    SoC_Value = "SoC_Value"
    meta = "meta"
    P_PV = "P_PV"
    P_Load = "P_Load"
    Q_Load = "Q_Load"
    mqtt = "mqtt"
    CarPark = "CarPark"

## CarPark

In [5]:
class CarPark:

    def __init__(self, charger_list, car_list):

        self.cars = {}
        self.chargers = {charger_index: charger for charger_index, charger in enumerate(charger_list)}
        self.vac_capacity = 0

        for car_index, car in enumerate(car_list):
            self.cars[car_index] = car
            self.vac_capacity += car.batteryCapacity

        self.number_of_cars = len(self.cars)

    def max_charge_power_calculator(self, charging_period):
        """
        Returns a dictionary "connections"
            keys: charger labels
            values: max charging power input to the connected car
        """

        connections = {}
        for key, charger in self.chargers.items():
            if charger.hosted_car:
                car = charger.hosted_car
                battery_depth_of_discharge = (1 - charger.soc) * car.batteryCapacity  # max_charging_power_kw-sec

                charger_limit = charger.kW
                car_limit = battery_depth_of_discharge / charging_period

                connections[key] = min(charger_limit, car_limit)

        return connections


## Car

In [6]:
class Car:

    def __init__(self, battery_capacity):
        self.batteryCapacity = battery_capacity * 3600

    def charge(self, soc, charge_period, charge_power):
        return soc + charge_power * charge_period / self.batteryCapacity


## ChargingStation

In [7]:
class ChargingStation:

    def __init__(self, max_charging_power_kw, hosted_car=None, soc=None):
        self.max_charging_power_kw = max_charging_power_kw

        if hosted_car:
            self.hosted_car = hosted_car
            self.plugged = True
            self.soc = soc
        else:
            self.plugged = False

    def plug(self, car):
        self.hosted_car = car
        self.plugged = True

    def unplug(self):
        self.hosted_car = None
        self.plugged = False

    def charge(self, charge_period, charge_power):

        if self.plugged == 1:
            self.soc = self.hosted_car.charge(self.soc, charge_period, charge_power)
        else:
            print("Charge decision cannot be implemented")


### Helper functions

## Input parser

In [13]:
class InputConfigParser:
    
    def __init__(self, input_config_file, input_config_mqtt, model_name):
        self.model_name = model_name
        self.model_variables = {'Feasible_ESS_Decisions': {'type': 'Set', 'indexing': 'None'}, 'Feasible_VAC_Decisions': {'type': 'Set', 'indexing': 'None'}, 'Value_Index': {'type': 'Set', 'indexing': 'None'}, 'Value': {'type': 'Param', 'indexing': 'index'}, 'P_PV_Forecast': {'type': 'Param', 'indexing': 'None'}, 'Initial_ESS_SoC': {'type': 'Param', 'indexing': 'index'}, 'Initial_VAC_SoC': {'type': 'Param', 'indexing': 'index'}, 'Number_of_Parked_Cars': {'type': 'Param', 'indexing': 'None'}, 'Unit_Consumption_Assumption': {'type': 'Param', 'indexing': 'None'}, 'Unit_Drop_Penalty': {'type': 'Param', 'indexing': 'None'}, 'ESS_Capacity': {'type': 'Param', 'indexing': 'None'}, 'VAC_Capacity': {'type': 'Param', 'indexing': 'None'}, 'Behavior_Model_Index': {'type': 'Set', 'indexing': 'None'}, 'Behavior_Model': {'type': 'Param', 'indexing': 'None'}, 'dT': {'type': 'Param', 'indexing': 'None'}, 'Decision': {'type': 'Var', 'indexing': 'index'}, 'P_ESS': {'type': 'Var', 'indexing': 'None'}, 'P_VAC': {'type': 'Var', 'indexing': 'None'}, 'P_PV': {'type': 'Var', 'indexing': 'index'}, 'P_GRID': {'type': 'Var', 'indexing': 'None'}}
        self.input_config_file = input_config_file
        self.input_config_mqtt = input_config_mqtt
        self.mqtt_params = {}
        self.generic_names = []
        self.generic_file_names = []
        
        self.defined_prediction_names = []
        self.defined_non_prediction_names = ["P_PV"]
        self.defined_external_names = ["SoC_Value"]
        self.prediction_names = []
        self.non_prediction_names = []
        self.external_names = []
        
        self.optimization_params = self.extract_optimization_values()
        
    def get_car_details(self, cars):
        pass

    def get_charger_details(self, chargers):
        pass
    
    def extract_optimization_values(self):
        data = {}
        for input_config in [self.input_config_file, self.input_config_mqtt]:
            for k, v in input_config.items():
                if isinstance(v, dict):
                    for k1, v1 in v.items():
                        if k1 == Constants.meta:
                            for k2, v2 in v1.items():
                                try:
                                    v2 = float(v2)
                                except ValueError:
                                    pass
                                if isinstance(v2, float) and v2.is_integer():
                                    v2 = int(v2)
                                if k2 in self.model_variables.keys():
                                    indexing = self.model_variables[k2]["indexing"]
                                    if indexing == "index":
                                        data[k2] = {int(0): v2}
                                    elif indexing == "None":
                                        data[k2] = {None: v2}
                                else:
                                    data[k2] = {None: v2}
                        elif k1 == Constants.SoC_Value and isinstance(v1, int):
                            indexing = self.model_variables[Constants.SoC_Value]["indexing"]
                            if indexing == "index":
                                data[Constants.SoC_Value] = {int(0): float(v1 / 100)}
                            elif indexing == "None":
                                data[Constants.SoC_Value] = {None: float(v1 / 100)}
                        elif isinstance(v1, list):
                            self.add_name_to_list(k1)
                        elif k == "generic" and not isinstance(v1, dict):
                            logger.debug("Generic single value")
                            try:
                                v1 = float(v1)
                            except ValueError:
                                pass
                            if isinstance(v1, float) and v1.is_integer():
                                v1 = int(v1)
                            data[k1] = {None: v1}
        return data
    